Data Source: [Heart Disease UCI](https://www.kaggle.com/ronitf/heart-disease-uci)

In [1]:
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns

import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
df = df.drop(columns=["cp", "restecg", "exang", "slope", "ca", "thal"])
df.iloc[0]

age          63.0
sex           1.0
trestbps    145.0
chol        233.0
fbs           1.0
thalach     150.0
oldpeak       2.3
target        1.0
Name: 0, dtype: float64

In [4]:
datas = df

count = np.zeros(9)

for i in range(303):
    data = datas.iloc[i]
    if data["age"] < 60:
            count[0] +=1
            
    else:
        if data["chol"] < 150:
            count[1] += 1
        else:
            count[2] += 1
            
print(count)

[213.   1.  89.   0.   0.   0.   0.   0.   0.]


In [5]:
datas = df
new = []

attrImpact = np.zeros(8)

for i in range(303):
    data = datas.iloc[i]
    if data["age"] < 55:
        if data["chol"] < 200:
            if data["fbs"] < 0.5:
                if data["trestbps"] < 200:
                    data["target"] = 0
                    attrImpact[0] += 1
                else:
                    data["target"] = 1
                    attrImpact[1] += 1
            else:
                data["target"] = 1
                attrImpact[2] += 1
        else:
            data["target"] = 1
            attrImpact[3] +=1
            
    else:
        if data["chol"] < 300:
            if data["fbs"] < 0.5:
                if data["trestbps"] < 160:
                    data["target"] = 0
                    attrImpact[4] += 1
                else:
                    data["target"] = 1
                    attrImpact[5] += 1
            else:
                data["target"] = 0
                attrImpact[6] += 1
        else:
            data["target"] = 1
            attrImpact[7] += 1
    new.append(data)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/

In [6]:
print(attrImpact)
print(attrImpact/303)

[ 26.   0.   3. 115.  93.  11.  25.  30.]
[0.08580858 0.         0.00990099 0.37953795 0.30693069 0.03630363
 0.08250825 0.0990099 ]


In [7]:
newDf = pd.DataFrame(new)
train = newDf.drop(columns=["target"])
y = newDf["target"]

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz

import pydotplus

In [9]:
dtree=DecisionTreeClassifier(max_depth=5)
dtree.fit(train,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(train),
                class_names=['no desease','have desease'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

In [10]:
dtree.feature_importances_


array([0.31537525, 0.        , 0.07654717, 0.51587229, 0.09220528,
       0.        , 0.        ])

# Original tree
![myTree2.jpeg](files/myTree2.jpeg)

# Generated tree
![myTree1.jpeg](files/myTree1.jpeg)



# My observation
### - The left bottom part of original tree
That part (node classified by "tresbps < 200") is not shown in generated tree. Since its gini = 0, which means it does not classify anything. Thus that classification(tresbps < 200) is useless, so it will not be shown in generated tree.

### - The right bottom part of original tree
"fbs" and "trestbps" node in original tree is switched in generated tree. Since information gain(ig) will decrease from 0.2496(original tree) to 0.1155(generated tree). So it's better to classify by trestbps first instead of fbs.

### - The other part of original tree
Looks similiar to other part of generated tree. Though there is little difference(e.g. gini of age node is 0.499 and 0.499 respectively), but it won't affect a lot.
